In [72]:
path = "localhost"
port = "5432"
user = "dragonlook"
password = "password"

dbname = "test"

cols = ['date',
 'time',
 'timezone',
 'username',
 'tweet',
 'language',
 'replies_count',
 'retweets_count',
 'likes_count',
 'hashtags']
pre_fix = ['date',
 'time',
 'timezone',
 'username',
 'language',
 'replies_count',
 'retweets_count',
 'likes_count']
pre_mod = ['tweet']
dtypes_pre = ['varchar(30)','varchar(30)','varchar(30)','varchar(300)','varchar','varchar(2)','integer','integer','integer','varchar(50)']
df_path = '/Users/dragonlook/Dropbox (MIT)/Private/System_Trade/Codes/MOTHER_DRAGON/NPL_dragon/lib/twint/STOCK_sentiment/ACN_mod.csv'

In [82]:
# create db
def create_db(cur, list_col, dname, dtypes=None, exist=None):
    drop_table = 'DROP TABLE '   + dname
    create_table = 'CREATE TABLE ' + dname + ' (' 
    for col, dtype in zip(list_col, dtypes):
        create_table = create_table + col + ' ' + dtype + ', '
        
    create_table = create_table[:-2] + ')'
    if exist:
        cur.execute(drop_table)
    cur.execute(create_table)

In [74]:
# cleansing data
import pandas as pd
df = pd.read_csv(df_path,usecols=cols)
# filter
df = df[df.loc[:,'language'] == 'en'] # lang filter
fil_ind = [ind for ind in range(len(df.loc[:,'hashtags'])) if len(df.loc[:,'hashtags'].to_list()[ind]) < 50 ]
df = df.iloc[fil_ind,:] # filter by hashtags
# count max vchar
max_vals = []
for pre_mod_col in pre_mod:
    max_vals.append(max([len(i) for i in df.loc[:,pre_mod_col]]))
for mod_col, max_val in zip(pre_mod, max_vals):
    ind = np.where(np.array(cols) == mod_col)[0][0]
    dtypes_pre[ind] = dtypes_pre[ind] + '(' + str(max_val) + ')'
df.to_csv('/Users/dragonlook/Dropbox (MIT)/Private/System_Trade/Codes/MOTHER_DRAGON/NPL_dragon/lib/twint/STOCK_sentiment/temp.csv')

In [83]:
# make db
import csv
import psycopg2
if password:
    conText = "host={} port={} dbname={} user={} password={}"
    conText = conText.format(path,port,dbname,user,password)
else:
    conText = "host={} port={} dbname={} user={}"
    conText = conText.format(path,port,dbname,user) 
connection = psycopg2.connect(conText)
connection.get_backend_pid()
cur = connection.cursor()
create_db(cur, cols, dbname, dtypes=dtypes_pre)

connection.commit()
connection.rollback()

with open('/Users/dragonlook/Dropbox (MIT)/Private/System_Trade/Codes/MOTHER_DRAGON/NPL_dragon/lib/twint/STOCK_sentiment/temp.csv', newline='') as csvfile:
    read = csv.reader(csvfile)
    cout = 0 
    for row in read:
        if cout == 0:
            cout += 1
            continue
        sql = "INSERT INTO " + dbname + " VALUES('{}',{})"
        sql = sql.format(str(row[0]).replace("/","-"),row[1])
        cur.execute(sql)
        connection.commit()
#         print(row)

cur.close()
connection.close()

['8', '6/10/21', '09:30:18', '-400.0', 'acn_canada1', 'Tomorrow we celebrate our benefactors! Thank you from all of us, especially our project partners! #BenefactorDays, #ACN  https://t.co/By3kQLW2RN', 'en', '0.0', '1.0', '2.0', "['benefactordays', 'acn']"]
['12', '6/10/21', '08:04:19', '-400.0', 'nadzombie', 'Thank you, Accenture C&amp;M Unplugged Team for inviting me as guest speaker on your #ACN  #CMuHomeBuddies #Ugnayan Podcast today ❤ hope everyone in the team had a fun, and informative day.  https://t.co/ohLXD7gLAy', 'en', '0.0', '0.0', '0.0', "['acn', 'cmuhomebuddies', 'ugnayan']"]
['33', '6/9/21', '04:23:54', '-400.0', 'options_buy', '#acn to buy  https://t.co/wszDu2mJ0X', 'en', '0.0', '0.0', '0.0', "['acn']"]
['45', '6/8/21', '05:51:27', '-400.0', 'activecn', 'We are delighted to join the @NISportsForum, working alongside sporting organisations to promote sport and physical recreation, with a view to improving physical and mental well-being!   #ACN 💜', 'en', '0.0', '0.0', '3.0

In [98]:
import lib.preprocess 
import importlib
PREPROCESS = importlib.reload(lib.preprocess).PREPROCESS
db = PREPROCESS('test','ACN_mod')

In [99]:
db.clensing()

In [100]:
db.create_spl_table('test_1',old=None)